## Create a calibration using pyFAI

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyFAI.gui.jupyter
import pyFAI
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
import pathlib
from tqdm import tqdm
import logging
import math
import warnings
import pyFAI.azimuthalIntegrator
from pyFAI.gui import jupyter
figure_size = 10

Load and show the calibration image

In [ ]:
filename = "../calibration/DLS_CeO2_1200mm.tif"
frame = fabio.open(filename).data
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(frame, ax=ax)

Define geometry components such as beam center and detector distance.

In [ ]:
x = 712.136 # x-coordinate of the beam-center in pixels
y = 727.864 # y-coordinate of the beam-center in pixels
d = 1194.046 # This is the distance in mm (unit used by Fit2d)
wl = 1.393000e-11 # Beam wavelength

Definition of the detector and of the calibrant

In [ ]:
detector = pyFAI.detectors.Detector(pixel1=0.000296, pixel2=0.000296)
print(detector)
calibrant = CALIBRANT_FACTORY("CeO2")
calibrant.wavelength = wl
calibrant

Set the geometry of the detector to be the guessed geometry

In [ ]:
initial_geometry = pyFAI.geometry.Geometry(detector=detector, wavelength=wl)
initial_geometry.setFit2D(d, x, y)
initial_geometry

We now use the SingleGeometry object to perform automatic ring extraction and calibration

In [ ]:
sg = SingleGeometry("demo", frame, calibrant=calibrant, detector=detector, geometry=initial_geometry)
sg.extract_cp(max_rings=16)

This figure shows that the fitted control points and rings overlap pretty well. Refinement probably won't change much.

In [ ]:
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(sg=sg, ax=ax)

We refine the geometry, we leave the wavelength fixed as we know this but allow the other parameters to change. This accounts for any slight inaccuracy in the detector set up.

In [ ]:
sg.geometry_refinement.refine2(fix=["wavelength"])

This figure shows the result of the refinement.

In [ ]:
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(sg=sg, ax=ax)

We can now save the refinement to a file to be used later

In [ ]:
# Delete the calibration if it already exists
calibration_path = "../calibration/DLS_CeO2_1200mm_pyFAI.poni"
# pathlib.Path(calibration_path).unlink(missing_ok=True) # only works for 3.8+
try:
    pathlib.Path(calibration_path).unlink()
except FileNotFoundError:
    pass

#Save the geometry obtained
sg.geometry_refinement.save(calibration_path)
with open(calibration_path) as f:
    print(f.read())

# Workflow for azimuthal integration and caking
First, we need to load the calibration from above, which contains information about our beamline setup like the orientation of the detector relative to the image. The calibration is stored in an `azimuthal integrator`.

In [ ]:
ai = pyFAI.load("../calibration/DLS_CeO2_1200mm_pyFAI.poni")
print("\nIntegrator: \n", ai)

# Azimuthal integration of multiple images for `TOPAS`

To perform an azimuthal integration we use the `integrate1d` function. 

There is also an `integrate2d` function, which is designed for caking of the data.

* The number of points in 2-theta is defined by the user.
* The azimuthal range runs from -180 to 180, or -pi to pi, rather than 0 to 360 as in DAWN, as explained in the section on [cake rotation](#cell_cake_rotation). An error will appear if using azimuth_range = (0,360).
* An output .dat file can be saved, which contains a header of metadata. 
* The result is returned as a numpy array of 2-theta and intensity.

An azimuthal integration can be performed like this. To iterate through multiple images we can create a loop, create an array for the data and then save it as a text file.

[TODO] - review all available options and determine those needed for our Diamond and DESY data.

In [ ]:
# suppress warnings when TIFFs are read
logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

# user inputs
number_of_points = 10000

# get a list of the files
image_list = sorted(pathlib.Path("../data/").glob("pixium*"))

for image_path in tqdm(image_list):
    # create an image array and integrate the data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        image = fabio.open(image_path)
    pattern_image_array = image.data
    result = ai.integrate1d(pattern_image_array,
                        number_of_points,
                        azimuth_range=(-180,180),
                        unit="2th_deg",
                        correctSolidAngle=True,
                        polarization_factor=0.99,
                        method='full_csr')
    
    result_array = np.column_stack((result.radial, result.intensity))
    
    # write out the integrated data to a text file
    np.savetxt(f"../analysis/azimuthal_integration_{image_path.stem}.xy", result_array)

# Caking multiple images for `xrdfit`

xrdfit has a simple data format with the first column being the bin edges of the two-theta angle and the subsequent
columns being the binned intensity data, one column for each cake.

Note that after the azimuthal integration the `np.flip` function is used to reverse the order of the columns of
intensity data. This means that the cakes are ordered clockwise rather than anticlockwise.

In [ ]:
# suppress warnings when TIFFs are read
logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

# user inputs
number_of_points = 10000
number_of_cakes = 36

# rotate the detector by half a cake width so that the cardinal direction
# is in the center of the first cake not the edge.
first_cake_angle = 360 / number_of_cakes
ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians

# get a list of the files
image_list = sorted(pathlib.Path("data/").glob("pixium*"))

for image_path in tqdm(image_list):
    # create empty array
    caked_data = np.zeros((number_of_cakes + 1, number_of_points))
    
    # create an image array and cake the data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        image = fabio.open(image_path)
    pattern_image_array = image.data
    result2d = ai.integrate2d(pattern_image_array,
                            number_of_points,
                            number_of_cakes,
                            unit="2th_deg",
                            polarization_factor=0.99,
                            method='full_csr')
    
    # flip the intensity data to order cakes clockwise rather than anticlockwise
    intensity = np.flip(result2d.intensity.T, axis=1)
    
    # reshape radial labels to 2D array so they can be attached to the intensity data.
    radial = np.reshape(result2d.radial, (-1, 1))
    
    result_array = np.hstack((radial, intensity))
    
    # write out the caked data to a text file
    np.savetxt(f"analysis/{image_path.stem}.dat", result_array)

This caked dataset is now saved in a format that we can use in [xrdfit](https://xrdfit.readthedocs.io/en/stable/) to
 analyse how the peak profiles change over time.
 
*Note, it is important to check that our input for `number_of_points` can fully resolve the peak profiles, we can check this by plotting the final cake.*

In [ ]:
plt.plot(result_array[:,0],result_array[:,2], marker = ".")
plt.xlim(3,4);

## Caking multiple images for `MAUD`

To cake data in preparation for analysis in [MAUD](http://maud.radiographema.eu) we need to change the output slightly to print out the intensity versus *pixel number*. PyFAI has the option of outputting intensity versus detector distance with the option *'unit = r_mm'*. We can then convert this value into a *pixel number*, by providing a pixel size in millimetres.

In [ ]:
# suppress warnings when TIFFs are read
logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

# user inputs
number_of_points = 10000
number_of_cakes = 72
pixel_size = 0.296

# rotate the detector by half a cake width so that the cardinal direction
# is in the center of the first cake not the edge.
first_cake_angle = 360 / number_of_cakes
ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians

# get a list of the files
image_list = sorted(pathlib.Path("data/MAUD/").glob("pixium*"))

for image_path in tqdm(image_list):
    # create empty array
    caked_data = np.zeros((number_of_cakes + 1, number_of_points))
    
    # create an image array and cake the data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        image = fabio.open(image_path)
    pattern_image_array = image.data
    result2d = ai.integrate2d(pattern_image_array,
                            number_of_points,
                            number_of_cakes,
                            unit="r_mm",
                            polarization_factor=0.99,
                            method='full_csr')
    
    # flip the intensity data to order cakes clockwise rather than anticlockwise
    intensity = np.flip(result2d.intensity.T, axis=1)
    
    # reshape radial labels to 2D array so they can be attached to the intensity data.
    radial_mm = np.reshape(result2d.radial, (-1, 1))
    radial_pixel = radial_mm/pixel_size
    
    result_array = np.hstack((radial_pixel, intensity))
    
    # write out the caked data to a text file
    np.savetxt(f"analysis/MAUD/{image_path.stem}.dat", result_array)

In [ ]:
plt.plot(result_array[:,0],result_array[:,2], marker = ".")
plt.xlim(210,260);